In [4]:
# The absolute value of the Fourier Transform of the loading distribution of a phased comb array of EMATs
# AND
# The ratio of in-plane to out-of-plane displacement for symmetric and antisymmetric modes at the surfaces of a plate, plotted as a 
# colour heatmap in phase velocity- frequency space
# [Written by Marcel Szczech during URSS project on lamb wave mode isolation]


                                    ########## What does this program do and why? ##########
# Firstly it plots the absolute value of the Fourier transform of the 'loading distribution' of a phased comb array of EMATs; equation 19.20
# in Chapter 19 of 'Ultrasonic Guided Waves in Solid Media' by J. L. Rose (2014). It allows you to see how the FT changes with number of
# EMATs, width of EMATs, spacing between EMATs (pitch) and time delay between adjacent EMATs emitting waves. The FT is plotted as a 
# heatmap in phase velocity - frequency space, where the colour represents the 'strength' of excitation for a given phase velocity and 
# frequency. The phase velocity - frequency curves for the modes possible for a given thickness plate are overlayed so that one can see
# where areas of higher excitation strength coincide with physical lamb wave modes. The reason why the FT of the loading distribution
# of the EMAT array corresponds to a pattern of excitation in phase-velocity - frequency space is because in Chapter 19 of the Rose 
# textbook mentioned earlier, it is stated that the amplitude of a produced mode is proportional to this FT (as well as other terms); see
# equation 19.1.

# Secondly it plots the ratio of in-plane to out-of-plane displacement for symmetric and antisymmetric modes, at the surfaces of a plate,
# as a heatmap in phase velocity - frequency space. This is useful because it places restrictions on what modes can be detected at 
# certain frequencies given you have a detector that is ONLY sensitive to in-plane or out-of-plane displacement; sometimes a mode that 
# seems to be excited for a particular driving current frequency according to the FT heatmap actually is not detected, or detected at 
# a lower intensity than expected, due to the fact that the in-plane and out-of-plane displacements for each mode vary as a function of 
# frequency.


                                            ########## The program itself ##########

# importing libraries.
# first line ensures no pop up graphs in other windows

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive
import seaborn as sb
from numpy.lib.scimath import sqrt


# Frequency domain in Hz
f = np.linspace(0.1,1250000,900)

# Phase velocity domain
cp = np.linspace(0.1,22000, 900)

# Create two 2D arrays, F and CP: 
# -> F will be an array of subarrays; EACH subarray will contain all the frequency points we'll evaluate the heatmap on 
#    (e.g. 0,30000, etc.)
# -> EACH of the subarrays is associated with a particular 'row' in cp-f space; the number of rows is the number of subarrays within F 
#    and is equal to the number of different cp values we consider in our domain 
# -> CP will contain subarrays; the first will be an array of the first value in the cp domain (0), as long as the length of a subarray 
#    in F. The second will be an array of the second value in the cp domain, etc. There are as many of these subarrays as you have values 
#    in your cp domain
# -> If you were to plot CP against F you'd have a grid of points for all possible combinations of f and cp defined above.
# -> np.meshgrid can be used to generate these two arrays

F, CP = np.meshgrid(f,cp)


# Check what they look like if you want

#print(F)
#print(CP)

# Values of a particular subarray in F plotted against the values in a subarray of CP, for all possible combinations of pairings 
# This bit is just to convince yourself of the nature of F and CP
# plt.plot(F, CP, marker='o', color='k', linestyle='none')
# plt.show()

# Reading in phase velocity vs frequency-thickness data up to 4 modes to plot on top of the heatmap. 
# The Rose textbook, and papers on mode isloation with phased comb transducers consider phase velocity rather than group velocity so
# I'll do the same. 

cp_S0 = np.loadtxt("Interpolated phase velocity curves.txt", delimiter=',', usecols=6)
cp_A0 = np.loadtxt("Interpolated phase velocity curves.txt", delimiter=',', usecols=1)

cp_S1 = np.loadtxt("Interpolated phase velocity curves.txt", delimiter=',', usecols=7)
cp_A1 = np.loadtxt("Interpolated phase velocity curves.txt", delimiter=',', usecols=2)

cp_S2 = np.loadtxt("Interpolated phase velocity curves.txt", delimiter=',', usecols=8)
cp_A2 = np.loadtxt("Interpolated phase velocity curves.txt", delimiter=',', usecols=3)

cp_S3 = np.loadtxt("Interpolated phase velocity curves.txt", delimiter=',', usecols=9)
cp_A3 = np.loadtxt("Interpolated phase velocity curves.txt", delimiter=',', usecols=4)

cp_S4 = np.loadtxt("Interpolated phase velocity curves.txt", delimiter=',', usecols=10)
cp_A4 = np.loadtxt("Interpolated phase velocity curves.txt", delimiter=',', usecols=5)

fd_cp = np.loadtxt("Interpolated phase velocity curves.txt", delimiter=',', usecols=0) # in MHz mm

# Transforming the frequency-thickness axis to frequency for a particular thickness plate; allows us to plot phase velocity against 
# frequency for a specified thickness plate.

thickness = 8.109 # in mm; the frequency thickness values are originally in MHz mm.

frequency = (fd_cp / thickness) * 10**(6) # in Hz

# Speed of longitudinal bulk waves in aluminium, m/s
# taken from David R. Lide, editor. CRC Handbook of Chemistry and Physics, 88th edition. Boca Raton, Florida: Taylor & Francis Group, 2008
c_L = 6420

# Speed of transverse bulk waves in aluminium, m/s
# taken from David R. Lide, editor. CRC Handbook of Chemistry and Physics, 88th edition. Boca Raton, Florida: Taylor & Francis Group, 2008
c_T = 3040

# Define the absolute value of the Fourier transform of the loading distribution of the phased comb EMAT array, in phase velocity 
# - frequency space 
# (equation 19.20 from J.L. Rose 2014 Ultrasonic Guided Waves in Solid Media)

def H(x,y,N,s,t0,w):
    return np.absolute( w*10**(-3) * ( np.sin( np.pi*x*w*10**(-3) / y) / ( np.pi*x*w*10**(-3) / y ) ) * ( (np.sin( N*( (np.pi*x*s*10**(-3)/y) - np.pi*x*t0*10**(-9) ) )) / (np.sin( (np.pi*x*s*10**(-3)/y) - np.pi*x*t0*10**(-9)) ) ) )

# Define the 2 functions giving the ratio of in-plane to out-of-plane displacement for symmetric and antisymmetric lamb wave modes. These
# are both done in one function. One of these (chosen by the user through one of the arguments) can be plotted alongside the |FT|
# of the loading distribution; together they can be used to determine to what strength a certain mode is excited, IF THE DETECTOR IS 
# SENSITIVE TO IN-PLANE or OUT-OF-PLANE DISPLACEMENT; this latter part is not taken into account by the FT of the loading distribution 
# alone.

def ratio(x,y,choice): # x is frequency, y is phase velocity
    # Section of code to calculate ratio of in-plane to out-of-plane displacement at the surface of the plate as frequency varies, for
    # symmetric and antisymmetric Lamb wave modes.

    # Theory:
    # The functions giving the in-plane and out-of-plane particle displacements, and stresses, as functions of mode frequency and plate 
    # depth are given in Chapter 6 of 'Ultrasonic Guided Waves in Solid Media' (2014), J. L. Rose. The functions giving the in-plane and
    # out-of-plane displacements contain unknown constants A_1, A_2, B_1 and B_2. I'll tackle the issue in the following way. I'll first 
    # set the two functions giving the in-plane stresses for symmetric and antisymmetric Lamb wave modes to 0; this will allow me to 
    # write the ratios R_1 = A_2 / B_1 and R_2 = A_1 / B_2 as functions of mode frequency and wavenumber. Next, I'll take the modulus of
    # the in-plane and out-of-plane displacement functions u and w for symmetric and antisymmetric lamb wave modes, and take the ratio 
    # |u|/|w| for symmetric and antisymmetric modes. The maths works out so that, for example in the symmetric |u|/|w| function, you can 
    # take out (B_1)^2 from inside the square root of the numerator and denominator, leaving the square roots with only terms in R_1, 
    # wavenumber and frequency, and the unknown B_1s cancel on top and bottom. This gives us leaves us with |u|/|w| solely as a function 
    # of mode frequency and wavenumber (since R_1 is a function of these too), so we can plot the function in phase velocity - frequency 
    # space.

    # There is one other key issue - because of the defintion of p and q, these can be imaginary. When we find |u|/|w| for symmetric or
    # antisymmetric modes, we'll see that we'll have to evaluate cos^2(ph), cos^2(qh), sin^2(ph) and sin^2(qh). The problem is that 
    # python can't handle complex arguments for trig functions. How do we get around this? First we realise that (and you can check 
    # yourself) that when c_T < c_p < c_L, p is imaginary, so for this regime we'll need alternate expressions for sin(ph) and cos(ph). 
    # When c_p < c_T, both p and q are imaginary so for this regime we'll need alternate expressions for sin(ph), sin(qh), cos(ph) and 
    # sin(ph). These alternate expressions will be what we code for first, and later we'll code for the symmetric mode and antisymmetric 
    # mode case.

    # First write if statements defining the regimes of c_p. The working for the alternate expressions for cos(ph) etc. is detailed in 
    # chapter 2.2 of the URSS summary!

    # Define the speed of bulk longitudinal and transverse waves in aluminium, c_L and c_T

    cL = 6420 # ms^-1
    cT = 3040 # ms^-1

    # Define the variables p, and q.

    p = 2*np.pi*x*sqrt( (1/cL)**2 - (1/y)**2 )
    q = 2*np.pi*x*sqrt( (1/cT)**2 - (1/y)**2 )

    # Define the 'depth' coordinate of the surface of the plate if the centre of the plate is at 0 depth; thickness/2

    h = (thickness/2)*10**(-3) # in metres, since thickness was in mm.

    # Define the wavenumber

    k = (2*np.pi*x) / (y)
        
    # Define the ratios R_1 and R_2

    R1 = ( (k**2 - q**2)*np.sin(q*h) ) / ( 2*1j*k*p*np.sin(p*h) )

    R2 = ( (k**2 - q**2)*np.cos(q*h) ) / ( -2*1j*k*p*np.cos(p*h) )

    # Define the ratio of in-plane to out-of plane particle displacement at the surface of the plate for symmetric and antisymmtric modes

    ratio_S = sqrt( ( (k*R1*np.cos(p*h))**2 + (q*np.cos(q*h))**2 )/( (p*R1*np.sin(p*h))**2 + (k*np.sin(q*h))**2 ) )

    ratio_A = sqrt( ( (k*R2*np.sin(p*h))**2 + (q*np.sin(q*h))**2 )/( (p*R2*np.cos(p*h))**2 + (k*np.cos(q*h))**2 ) )

    # Return the ratio for symmetric modes if 'choice' = 0, and the ratio for antisymmetric modes if 'choice' = 1.

    if choice == 0:
        return np.abs(ratio_S)

    if choice == 1:
        return np.abs(ratio_A)
        
# Defining the function which plots the |FT| function as a heatmap, with overlayed phase velocity vs frequency curves for modes.

def plotting_FT(N=4,s=6.5,t0=5000, w=2.5, freq_excite=0.6, phase_velocity=4): # The values set here are the default values for plotting
    z = H(F,CP,N,s,t0,w) # excitation spectrum values given a set of parameters
    ax = sb.heatmap(z, vmin=0, vmax=0.01) # heamtmap plotting; REMEMBER TO SET VMAX TO A SENSIBLE VALUE 

    ax.set_xlabel("Frequency (MHz)") 
    ax.set_ylabel("Phase velocity (m/s)")    

    # Plotting x and y axis ticks; the location of a tick is the index of the frequency value at which the tick is placed at, for the 
    # array of frequencies 'f'. The array f goes from 0 to 1.5 MHz

    # Defining the array of tick locations, expressed in indices of their respective value arrays
    xticks = np.linspace(0, len(f)-1, 6) # 6 ticks, starting on the 0th index frequency value (0 Hz), spaced equally apart, ending on the 
                                         # last index frequency value (1.5 MHz)
    yticks = np.linspace(0, len(cp)-1, 6)

    # plotting the ticks and labelling them to reflect physical values and not indices

    # x tick plotting and labelling 
    ax.set_xticks(xticks)
    ax.set_xticklabels(np.round(f[xticks.astype(int)] / 1e6, 2))  # Converts the index of a tick to the frequency it represents, rounding 
                                                                  # to 2 decimal places

    # y tick plotting and labelling
    ax.set_yticks(yticks)
    ax.set_yticklabels(np.round(cp[yticks.astype(int)], 1)) # Converts the index of a tick to the phase velocity it represents, rounding 
                                                            # to 2 decimal places      

    # Makes sure the y axis has ascending values. To do with how the tick labels are by default I think.
    ax.invert_yaxis()

    
    # Convert physical values to indices for plotting
    # np.interp(a,x,y) can be used to evaluate y values of an x-y data set, even for x values which may not originally be in the data set 
    # (interpolation), given by the argument a. Here we are treating the index arrays of the phase velocity and frequency arrays as our 
    # 'y' and the actual values of phase velocity and frequency that we'll plot over as 'x'. Then we are evaluating what the indices
    # corresponding to the phase velocity and frequency values in the phase-velocity vs frequency data for the modes WOULD be; the way
    # it is set up is that we have to plot indices not physical values.

    # Frequency indices
    frequency_idx = np.interp(frequency, f, np.arange(len(f))) 

    # Phase velocity indices
    cp_S0_idx = np.interp(cp_S0, cp, np.arange(len(cp))) 
    cp_A0_idx = np.interp(cp_A0, cp, np.arange(len(cp)))
    
    cp_S1_idx = np.interp(cp_S1, cp, np.arange(len(cp)))
    cp_A1_idx = np.interp(cp_A1, cp, np.arange(len(cp)))
    
    cp_S2_idx = np.interp(cp_S2, cp, np.arange(len(cp)))
    cp_A2_idx = np.interp(cp_A2, cp, np.arange(len(cp)))
    
    cp_S3_idx = np.interp(cp_S3, cp, np.arange(len(cp)))
    cp_A3_idx = np.interp(cp_A3, cp, np.arange(len(cp)))
    
    cp_S4_idx = np.interp(cp_S4, cp, np.arange(len(cp)))
    cp_A4_idx = np.interp(cp_A4, cp, np.arange(len(cp)))

    
    # Plotting the phase velocity vs frequency curves for up to 4 modes, for our chosen thickness
    # 'alpha' corresponds to the opacity of the line
    
    ax.plot(frequency_idx, cp_S0_idx, color='red', linewidth=2, alpha=0.8, label='S0')
    ax.plot(frequency_idx, cp_A0_idx, color='red', linewidth=2, alpha=0.4, linestyle='--', label='A0')
    
    ax.plot(frequency_idx, cp_S1_idx, color='blue', linewidth=2, alpha=0.8, label='S1')
    ax.plot(frequency_idx, cp_A1_idx, color='blue', linewidth=2, alpha=0.4, linestyle='--', label='A1')

    ax.plot(frequency_idx, cp_S2_idx, color='green', linewidth=2, alpha=0.8, label='S2')
    ax.plot(frequency_idx, cp_A2_idx, color='green', linewidth=2, alpha=0.4, linestyle='--', label='A2')

    ax.plot(frequency_idx, cp_S3_idx, color='yellow', linewidth=2, alpha=0.8, label='S3')
    ax.plot(frequency_idx, cp_A3_idx, color='yellow', linewidth=2, alpha=0.4, linestyle='--', label='A3')

    ax.plot(frequency_idx, cp_S4_idx, color='magenta', linewidth=2, alpha=0.8, label='S4')
    ax.plot(frequency_idx, cp_A4_idx, color='magenta', linewidth=2, alpha=0.4, linestyle='--', label='A4')

    # Plotting a vertical line for the frequency of the driving signal for identifying the 'line of excitation' due to a driving signal 
    # (freq_excite is in MHz)
    #ax.axvline( (len(f)/f[-1]) * freq_excite * 10**(6), color='white')

    # Plotting a horizontal line for a constant phase-velocity, chosen by the user. Just for visual aid.
    #ax.axhline( (len(cp)/cp[-1] * phase_velocity * 10**(3)), color='white')
    
    ax.legend(loc='center left', bbox_to_anchor=(1.3, 0.5), borderaxespad=0.)
    plt.show()





# Defining the function which PLOTS the in to out-of-plane displacement ratio function for symmetric or antisymmetric modes 
# as a heatmap, with overlayed phase-velocity vs frequency curves for modes.

def plotting_ratio(freq_excite=0.6, phase_velocity=4, choice=0): # These written values are just default plotting values
    z = ratio(F,CP,choice)
    ax = sb.heatmap(z, vmin=0, vmax=1) # heatmap plotting

    ax.set_xlabel("Frequency (MHz)") 
    ax.set_ylabel("Phase velocity (m/s)")    

    # Plotting x and y axis ticks; the location of a tick is the index of the frequency value at which the tick is placed at, for the 
    # array of frequencies 'f'. The array f goes from 0 to 1.5 MHz

    # Defining the array of tick locations, expressed in indices of their respective value arrays
    xticks = np.linspace(0, len(f)-1, 6) # 6 ticks, starting on the 0th index frequency value (0 Hz), spaced equally apart, ending on the 
                                         # last index frequency value (1.5 MHz)
    yticks = np.linspace(0, len(cp)-1, 6)


    # plotting the ticks and labelling them to reflect physical values and not indices

    # x tick plotting and labelling 
    ax.set_xticks(xticks)
    ax.set_xticklabels(np.round(f[xticks.astype(int)] / 1e6, 2))  # Converts the index of a tick to the frequency it represents, rounding 
                                                                  # to 2 decimal places

    # y tick plotting and labelling
    ax.set_yticks(yticks)
    ax.set_yticklabels(np.round(cp[yticks.astype(int)], 1)) # Converts the index of a tick to the phase velocity it represents, rounding 
                                                            # to 2 decimal places      

    # Makes sure the y axis has ascending values. To do with how the tick labels are by default I think.
    ax.invert_yaxis()

    
    # Convert physical values to indices for plotting

    # Frequency indices
    frequency_idx = np.interp(frequency, f, np.arange(len(f)))

    # Phase velocity indices
    cp_S0_idx = np.interp(cp_S0, cp, np.arange(len(cp))) 
    cp_A0_idx = np.interp(cp_A0, cp, np.arange(len(cp)))
    
    cp_S1_idx = np.interp(cp_S1, cp, np.arange(len(cp)))
    cp_A1_idx = np.interp(cp_A1, cp, np.arange(len(cp)))
    
    cp_S2_idx = np.interp(cp_S2, cp, np.arange(len(cp)))
    cp_A2_idx = np.interp(cp_A2, cp, np.arange(len(cp)))
    
    cp_S3_idx = np.interp(cp_S3, cp, np.arange(len(cp)))
    cp_A3_idx = np.interp(cp_A3, cp, np.arange(len(cp)))
    
    cp_S4_idx = np.interp(cp_S4, cp, np.arange(len(cp)))
    cp_A4_idx = np.interp(cp_A4, cp, np.arange(len(cp)))

    
    # Plotting the phase velocity vs frequency curves for up to 4 modes, for our chosen thickness
    
    ax.plot(frequency_idx, cp_S0_idx, color='red', linewidth=2, alpha=0.8, label='S0')
    ax.plot(frequency_idx, cp_A0_idx, color='red', linewidth=2, alpha=0.4, linestyle='--', label='A0')
    
    ax.plot(frequency_idx, cp_S1_idx, color='blue', linewidth=2, alpha=0.8, label='S1')
    ax.plot(frequency_idx, cp_A1_idx, color='blue', linewidth=2, alpha=0.4, linestyle='--', label='A1')

    ax.plot(frequency_idx, cp_S2_idx, color='green', linewidth=2, alpha=0.8, label='S2')
    ax.plot(frequency_idx, cp_A2_idx, color='green', linewidth=2, alpha=0.4, linestyle='--', label='A2')

    ax.plot(frequency_idx, cp_S3_idx, color='yellow', linewidth=2, alpha=0.8, label='S3')
    ax.plot(frequency_idx, cp_A3_idx, color='yellow', linewidth=2, alpha=0.4, linestyle='--', label='A3')

    ax.plot(frequency_idx, cp_S4_idx, color='magenta', linewidth=2, alpha=0.8, label='S4')
    ax.plot(frequency_idx, cp_A4_idx, color='magenta', linewidth=2, alpha=0.4, linestyle='--', label='A4')

    # Plotting a vertical line of frequency of the driving signal for identifying line of excitation due to driving signal 
    # (freq_excite is in MHz)
    #ax.axvline( (len(f)/f[-1]) * freq_excite * 10**(6), color='white')

    # Plotting a horizontal line for a constant phase-velocity, chosen by the user. Just a visual aid.
    #ax.axhline( (len(cp)/cp[-1] * phase_velocity * 10**(3)), color='white')
    
    ax.legend(loc='center left', bbox_to_anchor=(1.3, 0.5), borderaxespad=0.)
    plt.show()

# Calling the |FT| plotting function

interactive_plot_FT = interactive(plotting_FT, N=(1,50,1), s=(0.5,50,0.1), t0=(-10000, 10000, 50), w=(0,50,0.1), freq_excite=(0,3,0.01), phase_velocity=(0,22,0.01))
interactive_plot_FT

interactive(children=(IntSlider(value=4, description='N', max=50, min=1), FloatSlider(value=6.5, description='…

In [5]:
# Calling the plotting function for the ratio of in-plane to out-of-plane displacement for symmetric and antisymmetric modes
# Has to be done in a seperate cell due to the specifics of the code; I think it has to do with the fact that plt.show() should be called
# once at the end of all the plots; calling both the plotting functions for the displacements and the |FT| in one cell 
# would call plt.show() twice.

interactive_plot_ratio = interactive(plotting_ratio, freq_excite=(0,3,0.01), choice=(0,1,1), phase_velocity=(0,22,0.01))
interactive_plot_ratio

interactive(children=(FloatSlider(value=0.6, description='freq_excite', max=3.0, step=0.01), FloatSlider(value…